# What is lineslice?

Lineslice is a module that allows you to operate on a set of lines (a road network) in a dataframe like normal but allows you to include two fields: 'DISTANCES' & 'COLORKEYS' respectively to style and slice lines finer at some distance along each line or row in dataframe.

Both distances and colorkeys are stored as comma separted string in a pandas dataframe field. As you might have guessed we will have one more colorkey than distances as distances represent the junction or breaking nodes along the line. 

This is useful because you can operate using abstract data structures that take into account gradient as distance along a line then easily add that to an already existing data structure that can easily be sent out into a geojson. This makes it pretty useful for visualizations. (possibly)

# Requirements

Currently lineslice requires a dataframe to be formatted in a pretty specific way with a maxdistance field for each line and coords fields that is a string representation of what would go into a geojson alignment field.

If you would like to know more about how these dfs are formatted I suggest you look at [nlgeojson](https://github.com/murphy214/nlgeojson).

In [1]:
import lineslice
import mapkit as mk
import pandas as pd

# Reading a line representative dataframe into memory

As you can see its a typical df the important fields however are a uniqueidfield,coords, and maxdistance.

In [2]:
data = pd.read_csv('test.csv')[:30]
data

,gid,routeid,route,countycode,signsystem,subroute,label,suppcode,direction,dsurvey,...,labelrt,labelsrt,labelsup,district,shape_len,geom,coords,bounds,maxdistance,st_asewkt
0,25271,17400201300,20,17,4,13,20/13,0,SB,S,...,20,13.0,NaN,4.0,3252.725602,010500006025690000010000000102000040A800000060...,"[[-80.2991074977, 39.2279599544], [-80.2991919...","[[-80.3111116168, 39.2279599544], [-80.2991074...",0.032033,SRID=4326;MULTILINESTRINGM((-80.2991074976562 ...
1,25486,22400038200,3,22,4,82,3/82,0,NB,N,...,3,82.0,NaN,2.0,676.658895,0105000060256900000100000001020000402C00000060...,"[[-82.0868294566, 38.2827898199], [-82.0868036...","[[-82.0899223666, 38.2849238237], [-82.0858902...",0.007018,SRID=4326;MULTILINESTRINGM((-82.0868294565851 ...
2,14977,55400091700,9,55,4,17,9/17,0,EB,E,...,9,17.0,NaN,10.0,199.688657,0105000060256900000100000001020000400E00000070...,"[[-81.6016237413, 37.6634782282], [-81.6015410...","[[-81.6016237413, 37.6638740908], [-81.5994601...",0.002228,SRID=4326;MULTILINESTRINGM((-81.6016237413417 ...
3,21181,30400656300,65,30,4,63,65/63,0,NB,N,...,65,63.0,NaN,2.0,815.486855,0105000060256900000100000001020000402E00000070...,"[[-82.0908680586, 37.6887602916], [-82.0908873...","[[-82.0910738832, 37.6959284812], [-82.0906606...",0.007446,SRID=4326;MULTILINESTRINGM((-82.0908680586327 ...
4,25272,17400420000,42,17,4,0,42,0,EB,E,...,42,NaN,NaN,4.0,4318.688243,0105000060256900000100000001020000409800000060...,"[[-80.2991074977, 39.2279599544], [-80.2989727...","[[-80.2991074977, 39.2473361574], [-80.2681206...",0.044641,SRID=4326;MULTILINESTRINGM((-80.2991074976562 ...
5,25282,17400206900,20,17,4,69,20/69,0,EB,E,...,20,69.0,NaN,4.0,662.872144,0105000060256900000100000001020000403000000010...,"[[-80.3409389496, 39.380512371], [-80.34091945...","[[-80.3409389496, 39.380512371], [-80.33481413...",0.007262,SRID=4326;MULTILINESTRINGM((-80.3409389495564 ...
6,25283,17400204000,20,17,4,40,20/40,0,WB,W,...,20,40.0,NaN,4.0,303.243259,0105000060256900000100000001020000400A00000028...,"[[-80.3364518542, 39.38081834], [-80.336322186...","[[-80.3364518542, 39.3810813357], [-80.3333261...",0.003386,SRID=4326;MULTILINESTRINGM((-80.3364518541881 ...
7,25284,17400204100,20,17,4,41,20/41,0,NB,N,...,20,41.0,NaN,4.0,210.788758,01050000602569000001000000010200004005000000B0...,"[[-80.337553018, 39.3808085422], [-80.33761460...","[[-80.3378307202, 39.3826955676], [-80.3375530...",0.001907,SRID=4326;MULTILINESTRINGM((-80.3375530180092 ...
8,25285,17400206800,20,17,4,68,20/68,0,NB,N,...,20,68.0,NaN,4.0,189.610315,0105000060256900000100000001020000400F000000A8...,"[[-80.3384417225, 39.3807317465], [-80.3384610...","[[-80.3386966066, 39.3824277449], [-80.3384417...",0.001717,SRID=4326;MULTILINESTRINGM((-80.3384417225217 ...
9,25291,17400208600,20,17,4,86,20/86,0,SB,S,...,20,86.0,NaN,4.0,1022.897591,0105000060256900000100000001020000404D000000C0...,"[[-80.353003696, 39.3738589807], [-80.35300478...","[[-80.35300478, 39.3738589807], [-80.347964549...",0.010453,SRID=4326;MULTILINESTRINGM((-80.3530036960193 ...


# Creating a LineSlice Index

The underlying way that lineslice works is a dictionary like data structure for each line. To create a lineslice index that can be saved to disk I currently use h5 files to maintain df structure. To create the data structure call the following function. It also adds the colorkey field in an alternating fashion to definitively see the splits.

In [3]:
lineslice.make_lineslice_index(data,filename='example_lineslice_index.h5')

Making Split Index: [0 / 30]
Making Split Index: [1 / 30]
Making Split Index: [2 / 30]
Making Split Index: [3 / 30]
Making Split Index: [4 / 30]
Making Split Index: [5 / 30]
Making Split Index: [6 / 30]
Making Split Index: [7 / 30]
Making Split Index: [8 / 30]
Making Split Index: [9 / 30]
Making Split Index: [10 / 30]
Making Split Index: [11 / 30]
Making Split Index: [12 / 30]
Making Split Index: [13 / 30]
Making Split Index: [14 / 30]
Making Split Index: [15 / 30]
Making Split Index: [16 / 30]
Making Split Index: [17 / 30]
Making Split Index: [18 / 30]
Making Split Index: [19 / 30]
Making Split Index: [20 / 30]
Making Split Index: [21 / 30]
Making Split Index: [22 / 30]
Making Split Index: [23 / 30]
Making Split Index: [24 / 30]
Making Split Index: [25 / 30]
Making Split Index: [26 / 30]
Making Split Index: [27 / 30]
Making Split Index: [28 / 30]
Making Split Index: [29 / 30]
Wrote split_index h5 file to example_lineslice_index.h5.


''

# Reading the Lineslice index into memory 

To read the Index into memory call the function.



In [4]:
index = lineslice.read_lineslice_index('example_lineslice_index.h5')

# Creating a Sample Dataframe Input 

To create a sample input when applicable uniform_split_colors() accepts a dataframe argument like the one above and a size that each line will be split by, if lines are to small for any splits the data will be sliced to only contain applicable lines. The fields created can be seen below.

In [5]:
data = lineslice.uniform_split_colors(data,.001) 
data[['DISTANCES','COLORKEYS']]

,DISTANCES,COLORKEYS
0,"0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.00...","#2ADF05,#DB0F0A,#2ADF05,#DB0F0A,#2ADF05,#DB0F0..."
1,"0.001,0.002,0.003,0.004,0.005,0.006,0.007","#2ADF05,#DB0F0A,#2ADF05,#DB0F0A,#2ADF05,#DB0F0..."
2,"0.001,0.002","#2ADF05,#DB0F0A,#2ADF05"
3,"0.001,0.002,0.003,0.004,0.005,0.006,0.007","#2ADF05,#DB0F0A,#2ADF05,#DB0F0A,#2ADF05,#DB0F0..."
4,"0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.00...","#2ADF05,#DB0F0A,#2ADF05,#DB0F0A,#2ADF05,#DB0F0..."
5,"0.001,0.002,0.003,0.004,0.005,0.006,0.007","#2ADF05,#DB0F0A,#2ADF05,#DB0F0A,#2ADF05,#DB0F0..."
6,"0.001,0.002,0.003","#2ADF05,#DB0F0A,#2ADF05,#DB0F0A"
7,0.001,"#2ADF05,#DB0F0A"
8,0.001,"#2ADF05,#DB0F0A"
9,"0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.00...","#2ADF05,#DB0F0A,#2ADF05,#DB0F0A,#2ADF05,#DB0F0..."


# Finally creating the desired Dataframe output with split lines.

The final function to create a dataframe that can be sent into nlgeojson.make_lines() is the make_splits() function that accepts and a dataframe and a lineslice index. As you can see by the output the dataframes size is not much larger each row in the dataframe now represents one of the split segments of the line.

In [6]:
data = lineslice.make_splits(data,index)
data

,gid,ALIGNMENTS,routeid,route,countycode,signsystem,subroute,label,suppcode,direction,...,shape_len,geom,coords,bounds,maxdistance,st_asewkt,DISTANCES,COLORKEYS,a,COLORKEY
0,25271,"[[-80.2991919882,39.2278214803],[-80.299264826...",17400201300,20,17,4,13,20/13,0,SB,...,3252.725602,010500006025690000010000000102000040A800000060...,"[[-80.2991919882,39.2278214803],[-80.299264826...","[[-80.3111116168, 39.2279599544], [-80.2991074...",0.032033,SRID=4326;MULTILINESTRINGM((-80.2991074976562 ...,"0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.00...","#2ADF05,#DB0F0A,#2ADF05,#DB0F0A,#2ADF05,#DB0F0...",a,#2ADF05
1,25271,"[[-80.2997807584,39.2271386197],[-80.299866351...",17400201300,20,17,4,13,20/13,0,SB,...,3252.725602,010500006025690000010000000102000040A800000060...,"[[-80.2997807584,39.2271386197],[-80.299866351...","[[-80.3111116168, 39.2279599544], [-80.2991074...",0.032033,SRID=4326;MULTILINESTRINGM((-80.2991074976562 ...,"0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.00...","#2ADF05,#DB0F0A,#2ADF05,#DB0F0A,#2ADF05,#DB0F0...",a,#DB0F0A
2,25271,"[[-80.3007795116,39.2271960886],[-80.30099288,...",17400201300,20,17,4,13,20/13,0,SB,...,3252.725602,010500006025690000010000000102000040A800000060...,"[[-80.3007795116,39.2271960886],[-80.30099288,...","[[-80.3111116168, 39.2279599544], [-80.2991074...",0.032033,SRID=4326;MULTILINESTRINGM((-80.2991074976562 ...,"0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.00...","#2ADF05,#DB0F0A,#2ADF05,#DB0F0A,#2ADF05,#DB0F0...",a,#2ADF05
3,25271,"[[-80.3018488905,39.2273155748],[-80.302488154...",17400201300,20,17,4,13,20/13,0,SB,...,3252.725602,010500006025690000010000000102000040A800000060...,"[[-80.3018488905,39.2273155748],[-80.302488154...","[[-80.3111116168, 39.2279599544], [-80.2991074...",0.032033,SRID=4326;MULTILINESTRINGM((-80.2991074976562 ...,"0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.00...","#2ADF05,#DB0F0A,#2ADF05,#DB0F0A,#2ADF05,#DB0F0...",a,#DB0F0A
4,25271,"[[-80.3028090107,39.2273291767],[-80.303083376...",17400201300,20,17,4,13,20/13,0,SB,...,3252.725602,010500006025690000010000000102000040A800000060...,"[[-80.3028090107,39.2273291767],[-80.303083376...","[[-80.3111116168, 39.2279599544], [-80.2991074...",0.032033,SRID=4326;MULTILINESTRINGM((-80.2991074976562 ...,"0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.00...","#2ADF05,#DB0F0A,#2ADF05,#DB0F0A,#2ADF05,#DB0F0...",a,#2ADF05
5,25271,"[[-80.3038062257,39.2272655834],[-80.303995429...",17400201300,20,17,4,13,20/13,0,SB,...,3252.725602,010500006025690000010000000102000040A800000060...,"[[-80.3038062257,39.2272655834],[-80.303995429...","[[-80.3111116168, 39.2279599544], [-80.2991074...",0.032033,SRID=4326;MULTILINESTRINGM((-80.2991074976562 ...,"0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.00...","#2ADF05,#DB0F0A,#2ADF05,#DB0F0A,#2ADF05,#DB0F0...",a,#DB0F0A
6,25271,"[[-80.3047870626,39.2267678543],[-80.305120688...",17400201300,20,17,4,13,20/13,0,SB,...,3252.725602,010500006025690000010000000102000040A800000060...,"[[-80.3047870626,39.2267678543],[-80.305120688...","[[-80.3111116168, 39.2279599544], [-80.2991074...",0.032033,SRID=4326;MULTILINESTRINGM((-80.2991074976562 ...,"0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.00...","#2ADF05,#DB0F0A,#2ADF05,#DB0F0A,#2ADF05,#DB0F0...",a,#2ADF05
7,25271,"[[-80.3057908809,39.2261682327],[-80.306176977...",17400201300,20,17,4,13,20/13,0,SB,...,3252.725602,010500006025690000010000000102000040A800000060...,"[[-80.3057908809,39.2261682327],[-80.306176977...","[[-80.3111116168, 39.2279599544], [-80.2991074...",0.032033,SRID=4326;MULTILINESTRINGM((-80.2991074976562 ...,"0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.00...","#2ADF05,#DB0F0A,#2ADF05,#DB0F0A,#2ADF05,#DB0F0...",a,#DB0F0A
8,25271,"[[-80.306437115,39.225879553],[-80.3065737147,...",17400201300,20,17,4,13,20/13,0,SB,...,3252.725602,010500006025690000010000000102000040A800000060...,"[[-80.306437115,39.225879553],[-80.3065737147,...","[[-80.3111116168, 39.2279599544], [-80.2991074...",0.032033,SRID=4326;MULTILINESTRINGM((-80.2991074976562 ...,"0.001,0.

# Making Map

In [ ]:
#mk.make_lines(data,'',mask=True)
#mk.b()